In [8]:
#Import Packages

#To build a NN model
from keras.models import Sequential

#To build a densely connected NN layer
from keras.layers import Dense

#Regularization Function
from keras.layers import Dropout

#Activation Function
from keras.layers import Activation

#To build a fully connected RNN
from keras.layers import SimpleRNN

#Optimizer
from keras.optimizers import Adagrad

#To save keras model
from keras.models import load_model

#Math ops
import numpy as np

import os
import random

In [9]:
#Fetching Dataset
data = ''
files = os.listdir('../data/js/Basics/')
for file in files:
        data += open('../data/js/Basics/'+file,'r').read()

#Unique Characters
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'Total Characters: %d, Unique Characters: %d' % (data_size, vocab_size)

#Char-to_Index and Index-to-Char dicts
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

Total Characters: 217649, Unique Characters: 94


In [10]:
#Load saved model, if exists
if os.path.isfile('../dump/char-rnn-model.h5'):
    model = load_model('../dump/char-rnn-model.h5')
else:
    model = None

In [11]:
#Data Preprocessing
#Here we are preparing our data to model many-to-one model ie) our unrolled RNN contains many inputs and only one output  
time_steps = 30
if model is None:
    p = 0
    #Each list element contains time_steps characters of data
    inputs = []
    #Each list element contains corresponding character found at offset time_steps from data element
    targets = []
    while p+time_steps+1 <= data_size:
        #Convert time_steps characters into its integer representation
        inputs.append([char_to_ix[ch] for ch in data[p:p+time_steps]])
        #Convert target character into its integer representation
        targets.append(char_to_ix[data[p+time_steps]])
        p = p + 1


In [12]:
    #Convert data into a representation which is suited for our NN model
    #Here we are using one-hot vector representation which is an N dimension vector of all zeros except for a 1 at a corresponding integer index
    X = np.zeros((len(inputs), time_steps, vocab_size))
    y = np.zeros((len(targets), vocab_size))
    for i, inp in enumerate(inputs):
        for t, inp_t in enumerate(inp):
            X[i, t, inp_t] = 1
    for i, tgt in enumerate(targets):
            y[i, tgt] = 1

In [13]:
    #Build NN model
    model = Sequential()
    #Build RNN model
    #In this example
        #Input Layer contains time_steps * vocab_size of data ie) each row contains input at a particular time step 
        #and that input contains vocab_size of data in one-hot vector representation format
        #Hidden layer at each time step contains 100 (arbitrarily chosen) hidden units
        #Apply tanh activation function in hidden layer. 
    model.add(SimpleRNN(100, input_shape=(time_steps, vocab_size), activation='tanh'))
    
    #Dropout helps to prevent overfitting
    model.add(Dropout(0.1))
    
    #Using Dense add output layer which contains vocab_size output units - one per each unique character
    model.add(Dense(vocab_size))
    #Apply softmax activation function in output layer
    model.add(Activation('softmax'))
    model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 100)               19500     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 94)                9494      
_________________________________________________________________
activation_1 (Activation)    (None, 94)                0         
Total params: 28,994
Trainable params: 28,994
Non-trainable params: 0
_________________________________________________________________


**Model Parameters:** In our example total paramters is **28994**. Lets see how is this calculated.

Total Params = 28994 = 19500 + 9494

**19500** = (94 * 100) + (100 * 100) + 100

where 94 = Input Units, 100 = Hidden Units, 100 = Hidden Units, 100 = Hidden Units and 100 = Bias Units

**9494** = (100 * 94) + 94

where 100 = Hidden Units, 94 = Output Units and 94 = Bias Units

In [14]:
    #Optimizer
    optimizer = Adagrad(lr=0.01)
    #Configure the learning process.
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [15]:
    #Train the RNN model for 10 epochs and after training the model with a batch_size of 32 training samples, perform parameter update.
    model.fit(X, y, batch_size=32, epochs=10)
    #Save the trained model
    model.save('../dump/char-rnn-model.h5')

Epoch 1/10
217619/217619 [==============================] - 65s - loss: 1.7312    
Epoch 2/10
217619/217619 [==============================] - 60s - loss: 1.4308    
Epoch 3/10
217619/217619 [==============================] - 61s - loss: 1.3425    
Epoch 4/10
217619/217619 [==============================] - 61s - loss: 1.2903    
Epoch 5/10
217619/217619 [==============================] - 61s - loss: 1.2530    
Epoch 6/10
217619/217619 [==============================] - 61s - loss: 1.2252    
Epoch 7/10
217619/217619 [==============================] - 61s - loss: 1.2033    
Epoch 8/10
217619/217619 [==============================] - 61s - loss: 1.1858    
Epoch 9/10
217619/217619 [==============================] - 60s - loss: 1.1706    
Epoch 10/10
217619/217619 [==============================] - 59s - loss: 1.1584    


In [17]:
#Sequence Generation
for i in range(0,1):
    start_index = random.randint(0, data_size - time_steps - 1)
    idx = data.find('\n', start_index)
    if idx!=-1 and idx+time_steps <= data_size:
            sentence = data[idx: idx + time_steps]
    else:
        break
    sentence = data[idx: idx + time_steps]
    print '\nSeed:' + sentence
    print 'Generated Sentence:',
    #Generate next 200 characters for the given seed
    for i in range(200):
        x = np.zeros((1, time_steps, vocab_size))
        for t, char in enumerate(sentence):
            x[0, t, char_to_ix[char]] = 1.
        #Predict the target for the given sentence
        preds = model.predict(x, verbose=0)[0]
        #Find the index of the character whose probability is maximum
        next_index = np.argmax(preds)
        next_char = ix_to_char[next_index]
        print next_char,
        #Move input by one character
        sentence = sentence[1:] + next_char



Seed:
testProperty("nextSibling");
Generated Sentence: 
t e s t P r o p e r t y ( " s e r i n t " ) ; 
t e s t P r o p e r t y ( " c r i n g e t e t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e " ) ; 
t e s t P r o p e r t y ( " c o m a t e


Reference:

    1. https://gist.github.com/karpathy/d4dee566867f8291f086
    2. https://github.com/Microsoft/ChakraCore/tree/master/test/Basics